### Heroes Of Pymoli Data Analysis

-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)
purchase_data.head(3)

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88


## Player Count

* Display the total number of players


In [3]:
#total_number = pd.DataFrame({'Number of Players' : len(purchase_data['SN'].unique())})
#you would think this works, but it does not...
total_number = pd.DataFrame({'Number of Players' : pd.DataFrame((purchase_data['SN'].unique())).count()})
total_number

,Number of Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [4]:
pd.DataFrame({'NUI' : [len(purchase_data['Item ID'].unique())],'Mean Age of Clients (Y)' : [round(purchase_data['Age'].mean(), 1)],
'Age of Youngest Client (Y)' : [purchase_data['Age'].min()],
'Age of Eldest Client (Y)' : [purchase_data['Age'].max()],
'Average Price ($)' : [round(purchase_data['Price'].mean(), 1)],
'Total Sales ($)' : [round(purchase_data['Price'].sum(), 1)]})

#put all the relevant data into a dictionary and call pd.Dataframe on it. Keys set manually


,NUI,Mean Age of Clients (Y),Age of Youngest Client (Y),Age of Eldest Client (Y),Average Price ($),Total Sales ($)
0,183,22.7,7,45,3.1,2379.8


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [5]:
#group by gender and aggregate over SN by nunique: gives the unique SN per gender

grouped_by_gender = purchase_data.groupby(['Gender']).agg({'SN':'nunique'})

# extract the data from the aggregates
count_males = grouped_by_gender.loc['Male', 'SN']
count_females = grouped_by_gender.loc['Female', 'SN']
count_other = grouped_by_gender.loc['Other / Non-Disclosed', 'SN']

summed = grouped_by_gender['SN'].sum()
percentage_males = round(grouped_by_gender.loc['Male', 'SN'] / summed * 100, 1)
percentage_females = round(grouped_by_gender.loc['Female', 'SN'] / summed * 100, 1)
percentage_other = round(grouped_by_gender.loc['Other / Non-Disclosed', 'SN'] / summed * 100, 1)

# put it all together into a dictionary to make a dataframe
pd.DataFrame({'Gender' : ['Male', 'Female', 'Other / Non-Disclosed'], 'Count': [count_males, count_females, count_other], 'Percentage' : [percentage_males, percentage_females, percentage_other]}).set_index('Gender')

,Count,Percentage
Gender,,
Male,484,84.0
Female,81,14.1
Other / Non-Disclosed,11,1.9



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [11]:
# group by Gender, aggregate over SN by nunique, over Item ID by count(total number of items), over price by mean and gave a quick look at the result
grouped = purchase_data.groupby('Gender').agg({'SN' : 'nunique', 'Item ID' : 'count', 'Price' : 'mean'})
print(grouped.head())

# extract data using .loc, do the math, put all the resulting series to dataframe (), rename or set axis if necessary
purchase_count = grouped.loc[:, 'Item ID'].to_frame().rename(columns = {'Item ID' : 'Purchase Count'})
avg_price_gender = round(grouped.loc[:, 'Price'],1).to_frame().rename(columns = {'Price' : 'Average Purchase Price'})
total_value = round(grouped.loc[:, 'Item ID'] * grouped.loc[:, 'Price'], 1).to_frame().set_axis(['Total Purchases'], axis = 1)
avg_per_person = round(grouped.loc[:, 'Item ID'] * grouped.loc[:, 'Price']/grouped.loc[:,'SN'], 1).to_frame().set_axis(['Average Purchase per Person'], axis = 1)

# merge all
pd.merge(pd.merge(pd.merge(purchase_count, avg_price_gender, on = 'Gender'), total_value, on = 'Gender'), avg_per_person, on = 'Gender')

                        SN  Item ID     Price
Gender                                       
Female                  81      113  3.203009
Male                   484      652  3.017853
Other / Non-Disclosed   11       15  3.346000


,Purchase Count,Average Purchase Price,Total Purchases,Average Purchase per Person
Gender,,,,
Female,113,3.2,361.9,4.5
Male,652,3.0,1967.6,4.1
Other / Non-Disclosed,15,3.3,50.2,4.6


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [17]:
# create bins, we can safely assume everyone is younger than the Black Forest
mybins = [0, 9, 14, 19, 24, 29, 34, 39, 10000]

# drop duplicates
dropped = purchase_data.drop_duplicates(subset = 'SN')

# bin by age groups, get the size of each group, put to dataframe and set the index
binned = dropped.groupby(pd.cut(dropped['Age'], mybins)).size().to_frame()
myindex = pd.Series([' < 10', '10 - 14', '15 - 19', '20 - 24', '25 - 29', '30 - 34', '35 - 39', ' 40 < '])

count_only = binned.set_index(myindex).set_axis(['Count'], axis = 1)
percentages = round(count_only/count_only['Count'].sum()*100, 1).set_axis(['Percentage'], axis = 1)
count_only.join(percentages)

,Count,Percentage
< 10,17,3.0
10 - 14,22,3.8
15 - 19,107,18.6
20 - 24,258,44.8
25 - 29,77,13.4
30 - 34,52,9.0
35 - 39,31,5.4
40 <,12,2.1


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [9]:
mybins = [0, 9, 14, 19, 24, 29, 34, 39, 1000]
mylabels = [' < 10', '10 - 14', '15 - 19', '20 - 24', '25 - 29', '30 - 34', '35 - 39', ' 40 < ']

purchase_data['Age Groups'] = pd.cut(purchase_data['Age'], mybins, labels = mylabels)
purchase_dropped = purchase_data.drop(columns = ['Purchase ID', 'SN', 'Age', 'Item ID', 'Item Name'])
purchas_count = purchase_dropped.groupby(purchase_dropped['Age Groups']).size().to_frame().set_axis(['Purchase Count'], axis = 1)


mean_prices = round(purchase_dropped.groupby(purchase_dropped['Age Groups']).mean(),2).set_axis(['Average Purchase Value ($)'], axis = 1)

total_purchases = round(purchase_dropped.groupby(purchase_dropped['Age Groups']).sum(),2).set_axis(['Total Purchase Value ($)'], axis = 1)
avg_tpurch_perperson = round(total_purchases.loc[:,'Total Purchase Value ($)']/count_only.loc[:, 'Count'],2).to_frame().set_axis(['Average Total Purchase per Person ($)'], axis = 1)


# please note that the output examples gives incorrect answer to this: what is featured there is not per person, but per total count per age group, as some folks have multiple purchases

pd.merge(pd.merge(pd.merge(purchas_count, mean_prices, on = 'Age Groups'), total_purchases, on = 'Age Groups'), avg_tpurch_perperson, on = 'Age Groups')

,Purchase Count,Average Purchase Value ($),Total Purchase Value ($),Average Total Purchase per Person ($)
Age Groups,,,,
< 10,23,3.35,77.13,4.54
10 - 14,28,2.96,82.78,3.76
15 - 19,136,3.04,412.89,3.86
20 - 24,365,3.05,1114.06,4.32
25 - 29,101,2.90,293.00,3.81
30 - 34,73,2.93,214.00,4.12
35 - 39,41,3.60,147.67,4.76
40 <,13,2.94,38.24,3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [10]:
# drop unnecessary columns, find total speding per user and sort by price, rename column appropriately
# repeat for purchase count and average purchase value
# enough to pick first 5 of the dataframe used on the left of the left merge

purchase_updated = purchase_data.drop(columns = ['Purchase ID', 'Age Groups', 'Age', 'Item ID', 'Item Name', 'Gender'])
total_spending = purchase_updated.groupby(purchase_updated['SN']).sum().sort_values(by = 'Price', ascending = False).set_axis(['Total Purchase Value ($)'], axis = 1)
first_five = total_spending.iloc[0:5, :]
counts = purchase_updated.groupby(purchase_updated['SN']).count().set_axis(['Purchase Count'], axis = 1)
counts_merged = pd.merge(first_five, counts, on = 'SN', how = 'left')
avg_p_val = round(counts_merged.loc[:, 'Total Purchase Value ($)']/counts_merged.loc[:, 'Purchase Count'], 2).to_frame().set_axis(['Average Purchase Value ($)'], axis = 1)
pd.merge(counts_merged, avg_p_val, on = 'SN')




,Total Purchase Value ($),Purchase Count,Average Purchase Value ($)
SN,,,
Lisosia93,18.96,5,3.79
Idastidru52,15.45,4,3.86
Chamjask73,13.83,3,4.61
Iral74,13.62,4,3.40
Iskadarya95,13.10,3,4.37


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [11]:
# follow procedure from above

popular_items = purchase_data.groupby(purchase_data['Item ID']).count().sort_values(by = 'Purchase ID', ascending = False).drop(columns = ['Purchase ID', 'SN', 'Age', 'Price', 'Gender', 'Item Name']).set_axis(['Count'], axis = 1)
most_popular = popular_items.iloc[:5,:]
cleaned = purchase_data.set_index(['Item ID']).drop(columns = ['Purchase ID', 'SN', 'Age', 'Gender', 'Age Groups']).drop_duplicates().rename(columns = {'Price' : 'Price ($)'})
total_purchase_value = (popular_items.loc[:,'Count'] * cleaned.loc[:,'Price ($)']).to_frame().set_axis(['Total Purchase Value ($)'], axis = 1)
merged = pd.merge(pd.merge(popular_items, cleaned, how = 'left', left_index = True, right_index = True), total_purchase_value, how = 'left', left_index = True, right_index = True)


merged.iloc[:5,:]

,Count,Item Name,Price ($),Total Purchase Value ($)
Item ID,,,,
178,12,"Oathbreaker, Last Hope of the Breaking Storm",4.23,50.76
145,9,Fiery Glass Crusader,4.58,41.22
108,9,"Extraction, Quickblade Of Trembling Hands",3.53,31.77
82,9,Nirvana,4.90,44.10
19,8,"Pursuit, Cudgel of Necromancy",1.02,8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [12]:
merged.sort_values(['Total Purchase Value ($)'], ascending = False).head()

,Count,Item Name,Price ($),Total Purchase Value ($)
Item ID,,,,
178,12,"Oathbreaker, Last Hope of the Breaking Storm",4.23,50.76
82,9,Nirvana,4.90,44.10
145,9,Fiery Glass Crusader,4.58,41.22
92,8,Final Critic,4.88,39.04
103,8,Singed Scalpel,4.35,34.80
